In [2]:
import pandas as pd

In [18]:
df=pd.read_pickle("/workspace/LLaRA/data/ref/movielens/Processed_data.df")

In [19]:
df.head()

,seq,len_seq,next,movie_names_only,seq_only,movie_embeddings,most_similar_seq_index,most_similar_seq,most_similar_seq_next,most_similar_seq_name,most_similar_seq_next_name
0,"[(704, 5), (155, 3), (80, 4), (698, 4), (745, ...",10,"(3, 4)","[House of the Spirits, The (1993), Dirty Danci...","[704, 155, 80, 698, 745, 95, 1403, 202, 731, 432]","[-0.04578537, 0.050216097, -0.04009142, 0.0040...",4,"[(11, 5), (198, 5), (192, 5), (184, 5), (640, ...","(340, 3)","[Seven (Se7en) (1995), Nikita (La Femme Nikita...",Boogie Nights (1997)
1,"[(1305, 5), (135, 4), (1125, 5), (1124, 5), (2...",10,"(132, 5)","[National Lampoon's Senior Trip (1995), 2001: ...","[1305, 135, 1125, 1124, 240, 605, 477, 485, 52...","[-0.04719867, 0.017843362, -0.015032774, 0.013...",21,"[(1049, 3), (661, 4), (615, 4), (181, 4), (503...","(229, 4)","[House Arrest (1996), High Noon (1952), 39 Ste...",Star Trek III: The Search for Spock (1984)
2,"[(1030, 5), (394, 3), (104, 3), (779, 4), (103...",10,"(671, 3)","[Beverly Hillbillies, The (1993), Radioland Mu...","[1030, 394, 104, 779, 1036, 66, 446, 1029, 451...","[-0.06387466, 0.02771371, -0.03948658, -0.0078...",5,"[(21, 5), (10, 3), (422, 3), (27, 4), (317, 4)...","(154, 5)","[Muppet Treasure Island (1996), Richard III (1...",Monty Python's Life of Brian (1979)
3,"[(537, 4), (293, 4), (244, 3), (287, 5), (524,...",10,"(21, 5)","[My Own Private Idaho (1991), Donnie Brasco (1...","[537, 293, 244, 287, 524, 514, 922, 602, 172, 97]","[-0.040997818, 0.014861169, -0.032643072, -0.0...",25,"[(90, 3), (240, 3), (486, 5), (171, 4), (514, ...","(834, 5)","[So I Married an Axe Murderer (1993), Beavis a...",Halloween: The Curse of Michael Myers (1995)
4,"[(11, 5), (198, 5), (192, 5), (184, 5), (640, ...",10,"(340, 3)","[Seven (Se7en) (1995), Nikita (La Femme Nikita...","[11, 198, 192, 184, 640, 581, 653, 432, 356, 666]","[-0.05878972, 0.032049313, -0.033602327, 0.013...",0,"[(704, 5), (155, 3), (80, 4), (698, 4), (745, ...","(3, 4)","[House of the Spirits, The (1993), Dirty Danci...",Four Rooms (1995)


In [22]:
df['movie_names_only'][1]

["National Lampoon's Senior Trip (1995)",
 '2001: A Space Odyssey (1968)',
 'Innocents, The (1961)',
 'Farewell to Arms, A (1932)',
 'Beavis and Butt-head Do America (1996)',
 'Meet Me in St. Louis (1944)',
 'Matilda (1996)',
 'My Fair Lady (1964)',
 'My Life as a Dog (Mitt liv som hund) (1985)',
 'Forbidden Planet (1956)']

In [23]:
df['most_similar_seq_name'][1]

['House Arrest (1996)',
 'High Noon (1952)',
 '39 Steps, The (1935)',
 'Return of the Jedi (1983)',
 'Candidate, The (1972)',
 'Laura (1944)',
 'Big Sleep, The (1946)',
 "Breakfast at Tiffany's (1961)",
 'Under Siege (1992)',
 'Made in America (1993)']

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95 entries, 0 to 94
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   seq      95 non-null     object
 1   len_seq  95 non-null     int64 
 2   next     95 non-null     object
dtypes: int64(1), object(2)
memory usage: 2.4+ KB


In [15]:
import pickle
import pandas as pd
import numpy as np
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean
import torch
from sentence_transformers import SentenceTransformer

# 全局加载模型
model = SentenceTransformer(model_name_or_path="/workspace/all-MiniLM-L6-v2/")

In [16]:
def load_data(file_path):
    return pd.read_pickle(file_path)

def load_movie_dict(item_file):
    item_df = pd.read_csv(item_file, sep='|', header=None, encoding='latin-1', usecols=[0, 1])
    item_df.columns = ['movie_id', 'movie_title']
    movie_dict = dict(zip(item_df['movie_id'], item_df['movie_title']))
    return movie_dict

def map_movie_names_only(seq, movie_dict):
    return [movie_dict[id] if id in movie_dict else id for (id, rating) in seq]

def extract_sequences(df, movie_dict):
    df['movie_names_only'] = df['seq'].apply(lambda x: map_movie_names_only(x, movie_dict))
    df['seq_only'] = df['seq'].apply(lambda x: [id for (id, rating) in x])
    return df

def get_movie_embeddings(movie_list):
    embeddings = []
    for movies in movie_list:
        movie_strings = [str(movie) for movie in movies]
        movie_embeddings = model.encode(movie_strings)
        sequence_embedding = np.mean(movie_embeddings, axis=0)  # 平均化得到一维 embedding
        embeddings.append(sequence_embedding)
    return np.array(embeddings)

def calculate_dtw_distance(embedding_seq1, embedding_seq2):
   
    distance, path = fastdtw(embedding_seq1.reshape(-1, 1), embedding_seq2.reshape(-1, 1), dist=euclidean)
    return distance

def calculate_similarity(df):
    movie_embeddings = get_movie_embeddings(df['movie_names_only'].tolist())
    df['movie_embeddings'] = list(movie_embeddings)
    embeddings = np.stack(df['movie_embeddings'].values)
    
    most_similar_indices = []
    for i, embedding_seq1 in enumerate(embeddings):
        min_distance = float('inf')
        most_similar_index = -1
        for j, embedding_seq2 in enumerate(embeddings):
            if i != j:
                distance = calculate_dtw_distance(embedding_seq1, embedding_seq2)
                if distance < min_distance:
                    min_distance = distance
                    most_similar_index = j
        most_similar_indices.append(most_similar_index)
    
    df['most_similar_seq_index'] = most_similar_indices
    df['most_similar_seq'] = df['most_similar_seq_index'].apply(lambda idx: df.at[idx, 'seq'])
    return df

def add_most_similar_seq_next(df, movie_dict):
    df['most_similar_seq_next'] = df['next'].iloc[df['most_similar_seq_index']].values
    df['most_similar_seq_name'] = df['most_similar_seq'].apply(lambda x: [movie_dict.get(item[0], "Unknown") for item in x])
    df['most_similar_seq_next_name'] = df['most_similar_seq_next'].apply(lambda x: movie_dict.get(x[0], "Unknown"))
    return df

def save_data(df, output_file_path):
    df.to_pickle(output_file_path)

def process_data(file_path, item_file, output_file_path):
    df = load_data(file_path)
    movie_dict = load_movie_dict(item_file)
    df = extract_sequences(df, movie_dict)
    df = calculate_similarity(df)
    df = add_most_similar_seq_next(df, movie_dict)
    save_data(df, output_file_path)


In [17]:
# 使用示例
file_path = "/workspace/LLaRA/data/ref/movielens/Test_data.df"
item_file = "/workspace/LLaRA/data/ref/movielens/u.item"
output_file_path = "/workspace/LLaRA/data/ref/movielens/Processed_data.df"
process_data(file_path, item_file, output_file_path)


Shape of embedding_seq1: (384,)
Shape of embedding_seq2: (384,)
Shape of embedding_seq1: (384,)
Shape of embedding_seq2: (384,)
Shape of embedding_seq1: (384,)
Shape of embedding_seq2: (384,)
Shape of embedding_seq1: (384,)
Shape of embedding_seq2: (384,)
Shape of embedding_seq1: (384,)
Shape of embedding_seq2: (384,)
Shape of embedding_seq1: (384,)
Shape of embedding_seq2: (384,)
Shape of embedding_seq1: (384,)
Shape of embedding_seq2: (384,)
Shape of embedding_seq1: (384,)
Shape of embedding_seq2: (384,)
Shape of embedding_seq1: (384,)
Shape of embedding_seq2: (384,)
Shape of embedding_seq1: (384,)
Shape of embedding_seq2: (384,)
Shape of embedding_seq1: (384,)
Shape of embedding_seq2: (384,)
Shape of embedding_seq1: (384,)
Shape of embedding_seq2: (384,)
Shape of embedding_seq1: (384,)
Shape of embedding_seq2: (384,)
Shape of embedding_seq1: (384,)
Shape of embedding_seq2: (384,)
Shape of embedding_seq1: (384,)
Shape of embedding_seq2: (384,)
Shape of embedding_seq1: (384,)
Shape of